# Learn OpenAI Whisper - Chapter 5
## Notebook 1: Setting up Whisper for transcription

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1j1hoGvoOfZkh_p8dKceC4lS895YZKs9k)

This notebook provides a guide for setting up and using OpenAI's Whisper model for audio transcription tasks.

I hope you find this guide both informative and practical, setting you up for success in your audio transcription endeavors. If you have any questions or need further clarification on any of the steps, feel free to reach out.

Here's a high-level summary of its contents, along with suggested titles for code sections:

1. **Setting up the environment and installing dependencies**
2. **Downloading audio files**
3. **Verifying of compute resources**
4. **Loading the Whisper model**
5. **Setting the Natural Language Toolkit (NLTK) library**
6. **Transcribing and language detection**
7. **Defining a transcription function**
8. **Transcribing non-English audio with optional translation**
9. **Enhancing transcription with custom options**
   
Through these steps, I aim to equip you with a comprehensive understanding of Whisper's transcription and translation capabilities. By applying these techniques, you'll be well-prepared to tackle a wide range of audio processing tasks, from simple transcriptions to more complex, multilingual projects.

1. **Setting up the environment and installing dependencies**
   
   This section contains commands to install necessary dependencies and packages such as ffmpeg for handling audio files, Python libraries for machine learning (like cohere, openai, tiktoken), and the Whisper library directly from its GitHub repository.

In [ ]:
%%capture
!sudo apt install ffmpeg
!pip install -q cohere openai tiktoken
!pip install -q git+https://github.com/openai/whisper.git

2. **Downloading audio files**
   
    Here, we download sample audio files from a GitHub repository and the OpenAI's content delivery network or CDN. It includes both English and Spanish audio samples, preparing the ground for transcription tasks.


In [ ]:
!wget -nv https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter01/Learn_OAI_Whisper_Sample_Audio01.mp3
!wget -nv https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter01/Learn_OAI_Whisper_Spanish_Sample_Audio01.mp3
!wget -nv https://cdn.openai.com/API/examples/data/bbq_plans.wav
!wget -nv https://cdn.openai.com/API/examples/data/product_names.wav

audiofiles = ['Learn_OAI_Whisper_Sample_Audio01.mp3', 'Learn_OAI_Whisper_Spanish_Sample_Audio01.mp3', 'bbq_plans.wav', 'product_names.wav']


3. **Verifying of compute resources**

    This cell checks for the availability of a GPU, which is crucial for running the Whisper model efficiently. It sets the device to either CPU or GPU based on the availability, ensuring optimal performance.

    Using a GPU is the preferred way to use Whisper. If you are using a local machine, you can check if you have a GPU available. The first line results `False`, if Cuda compatible Nvidia GPU is not available and `True` if it is available. The second line of code sets the model to preference GPU whenever it is available.

In [ ]:
import numpy as np
import torch
torch.cuda.is_available()
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using torch {torch.__version__} ({DEVICE})")

4. **Loading the Whisper model**

    This section details the process of loading the Whisper model with a specific configuration (in this case, "medium") and checking its language support and parameter count. It's essential for understanding the model's capabilities and requirements.

    There are several Whisper models available. A list of them is [here](https://github.com/openai/whisper/blob/main/model-card.md). Each model has pros and cons in terms of accuracy and speed (compute needed). We will use the `medium` model for this tutorial.

In [ ]:
import whisper
model = whisper.load_model("medium", device=DEVICE)
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

5. **Setting the Natural Language Toolkit (NLTK) library**

    Before processing transcriptions, this part includes setup steps for NLTK, a toolkit in Python for working with human language data. It's used here to enhance the readability of transcriptions by sentence segmentation.


In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize

6. **Transcribing and language detection**

    For each file, we first ensure it's in a format Whisper can process efficiently. Then, we leverage Whisper's capability to detect the language of the audio. This step is crucial for setting up our transcription tasks with the correct language, ensuring higher accuracy. We decode the audio and present the transcription in a readable format, sentence by sentence.

    **Do you need to set `fp16` true or false?**
    - **On GPU:** Setting `fp16` to `True` can be beneficial if your GPU supports half-precision computations. This is often the case with modern NVIDIA GPUs that support Tensor Cores (e.g., Volta, Turing, Ampere, and newer architectures). It can reduce memory usage and speed up model inference.
    - **On CPU:** You should set `fp16` to `False`, as `fp16` computations are generally not supported or not efficient on CPUs. This avoids compatibility issues and ensures your code runs smoothly.

    Using this dynamic approach allows your script to automatically adjust to the execution environment, making it more versatile and user-friendly, especially if you plan to run it across different hardware setups.

In [ ]:
for audiofile in audiofiles:
    # Load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audiofile)
    audio = whisper.pad_or_trim(audio)
    # Make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    #Next we detects the language of your audio file
    _, probs = model.detect_language(mel)
    detected_language = max(probs, key=probs.get)
    print(f"----\nDetected language: {detected_language}")
    # Set up the decoding options
    options = whisper.DecodingOptions(language=detected_language, without_timestamps=True, fp16=(DEVICE == "cuda"))
    # Decode the audio and print the recognized text
    result = whisper.decode(model, mel, options)
    print("Transcription of file '" + audiofile + "':")
    for sent in sent_tokenize(result.text):
        print(sent)

7. **Defining a transcription function**

    To streamline our workflow, we define a function that handles the loading of audio files, sets transcription and translation options, and performs the transcription. This function is versatile, allowing us to specify whether we want translation alongside transcription, making it a handy tool for processing multiple files.

In [ ]:
def process_file(audiofile, model, w_options, w_translate=False):
    # Load audio
    audio = whisper.load_audio(audiofile)
    transcribe_options = dict(task="transcribe", **w_options)
    translate_options = dict(task="translate", **w_options)
    transcription = model.transcribe(audiofile, **transcribe_options)["text"]
    # Evaluates whether a translation is requested
    if w_translate:
        translation = model.transcribe(audiofile, **translate_options)["text"]
    else:
        translation = "N/A"

    return transcription, translation

8. **Transcribing non-English audio with optional translation**

    Here, we apply our transcription function to a Spanish audio sample, demonstrating Whisper's multilingual capabilities. We show how to obtain both the transcription and, optionally, the translation, showcasing the model's utility in handling diverse languages.
    
    We introduce an interactive element that allows you to listen to the audio files directly within the notebook. This feature enhances the user experience, providing an immediate sense of the audio content being transcribed and translated.

In [ ]:
w_options = dict(without_timestamps=True, fp16=(DEVICE == "cuda"))
audiofile = 'Learn_OAI_Whisper_Spanish_Sample_Audio01.mp3'
transcription, translation = process_file(audiofile, model, w_options, False)

print("------\nTranscription of file '" + audiofile + "':")
for sent in sent_tokenize(transcription):
    print(sent)
print("------\nTranslation of file '" + audiofile + "':")
for sent in sent_tokenize(translation):
    print(sent)

import ipywidgets as widgets
widgets.Audio.from_file(audiofile, autoplay=False, loop=False)

9. **Enhancing transcription with custom options**

    We explore how to tailor the transcription process further using custom options, such as controlling the decoding temperature and providing an initial prompt. This allows us to fine-tune the transcription results, adapting to specific audio content like product names or technical terminology.

9.1. **Using a spelling guide**

Let's tackle a common challenge when working with Whisper: ensuring it accurately transcribes uncommon proper nouns, such as product names, company names, or individuals. These elements can often trip up even the most sophisticated transcription tools, leading to misspellings.

In [ ]:
w_options = dict(without_timestamps=True, fp16=(DEVICE == "cuda"), temperature=0, initial_prompt="")
audiofile = 'product_names.wav'
transcription, translation = process_file(audiofile, model, w_options)

print("------\nTranscription of file '" + audiofile + "':")
for sent in sent_tokenize(transcription):
    print(sent)

import ipywidgets as widgets
widgets.Audio.from_file(audiofile, autoplay=False, loop=False)

To guide Whisper towards our preferred spellings, we'll include these names directly in the prompt, essentially creating a glossary for Whisper to reference.

In [ ]:
w_options = dict(without_timestamps=True, fp16=(DEVICE == "cuda"), temperature=0, initial_prompt="Quirk Quid Quill Inc., P3-Quattro, O3-Omni, B3-BondX, E3-Equity, W3-WrapZ, O2-Outlier, U3-UniFund, M3-Mover")
audiofile = 'product_names.wav'
transcription, translation = process_file(audiofile, model, w_options)

print("------\nTranscription of file '" + audiofile + "':")
for sent in sent_tokenize(transcription):
    print(sent)

**9.2 Prompting for transcript generation**

Next, we'll pivot to a different audio clip crafted specifically for this exercise. The scenario? An unusual barbecue event. Our first step involves generating a baseline transcript with Whisper to assess its initial accuracy.

In [ ]:
w_options = dict(without_timestamps=True, fp16=(DEVICE == "cuda"), temperature=0, initial_prompt="")
audiofile = 'bbq_plans.wav'
transcription, translation = process_file(audiofile, model, w_options)

print("------\nTranscription of file '" + audiofile + "':")
for sent in sent_tokenize(transcription):
    print(sent)

import ipywidgets as widgets
widgets.Audio.from_file(audiofile, autoplay=False, loop=False)

While Whisper does an impressive job with transcription, it sometimes has to make educated guesses about spellings. Take, for instance, it rendering the names as Amy and Sean, when in fact, they're spelled Aimee and Shawn. Our goal now is to see if we can influence these spellings through a carefully crafted prompt.

In [ ]:
w_options = dict(without_timestamps=True, fp16=(DEVICE == "cuda"), temperature=0, initial_prompt="Friends: Aimee, Shawn")
audiofile = 'bbq_plans.wav'
transcription, translation = process_file(audiofile, model, w_options)

print("------\nTranscription of file '" + audiofile + "':")
for sent in sent_tokenize(transcription):
    print(sent)

Now, let's apply "prompting for transcript generation" convert instructions into fictitious transcripts for Whisper to emulate. Using that technique to other words with tricky spellings, we ensure our transcript is as accurate as possible in every detail.

In [ ]:
w_options = dict(without_timestamps=True, fp16=(DEVICE == "cuda"), temperature=0, initial_prompt=""""Aimee and Shawn had whisky, doughnuts, omelets at a BBQ.""")
audiofile = 'bbq_plans.wav'
transcription, translation = process_file(audiofile, model, w_options)

print("------\nTranscription of file '" + audiofile + "':")
for sent in sent_tokenize(transcription):
    print(sent)